In [1]:
import os

In [2]:
os.getcwd()

'e:\\Mlop\\End to end\\End-to-end-Detecting-Card-Fraud\\research'

In [3]:
os.chdir("../")

In [ ]:
os.getcwd()




'e:\\Mlop\\End to end\\End-to-end-Detecting-Card-Fraud'

In [65]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    data_source: Path
    processed_data_dir: Path
    train_file: Path
    test_file: Path

In [66]:
from src.CardFraud.constant import*
from src.CardFraud.utils.common import read_yaml , create_directories

In [68]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath =CONFIG_FILE_PATH,
        params_filepath =PARAMS_FILE_PATH
    ):
        
        self.config =read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
    
        data_ingestion_config = DataIngestionConfig(
            data_source=self.config.data_ingestion.data_source,
            processed_data_dir=self.config.data_ingestion.processed_data_dir,
            train_file=self.config.data_ingestion.train_file,
            test_file=self.config.data_ingestion.test_file
        )
        
        
        return data_ingestion_config

In [54]:
config_filepath =CONFIG_FILE_PATH
config =read_yaml(config_filepath)

In [57]:
config.data_ingestion.root_dir

'artifacts/data_ingestion'

In [69]:
import pandas as pd
from exception import CustomException
from logger import logging
import sys
from sklearn.model_selection import train_test_split
class DataIngestion:
    def __init__(self , config:DataIngestionConfig):
        self.config = config
        
    def read_data(self)-> pd.DataFrame:
        
        try:
            data=pd.read_csv(self.config.data_source)
            logging.info("data Read succssfully")
            return data
        except Exception as e:
            raise CustomException(e, sys)
        
        
    def save_data(self, data:pd.DataFrame , filepath:Path):
        os.makedirs(os.path.dirname(filepath),exist_ok=True)
        data.to_csv(filepath, index=False)
        logging.info("data Saved succssfully")
    
    def split_data(self, data:pd.DataFrame):
        try:
            
        
            train_data,test_data =train_test_split(data , test_size=0.2 , random_state=42)
            
            self.save_data(train_data ,os.path.join(self.config.processed_data_dir,self.config.train_file ))
            self.save_data(test_data ,os.path.join(self.config.processed_data_dir,self.config.test_file ))
            logging.info("data Split succssfully")
        except Exception as e:
            raise CustomException(e, sys)
        
        
    def initiate_data_ingestion(self)->None:
        
        
        data =self.read_data()
        self.split_data(data)
        
        
            
        

In [70]:
try:
    config = ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
    raise CustomException